# Show all detections around masks on display
Quick notebook to make plots/visual inspections similar to the last image of: https://hsc-release.mtk.nao.ac.jp/doc/index.php/known-problems-2/#hsc-link-2

Change next cell to select the patch to show and detections to keep, then run everything:

In [1]:
import numpy as np

chosenFilter = "HSC-R" 
tract = 9697
# random patch among those in my last minute run
patch = "{},{}".format(np.random.choice(range(2,7),1)[0], np.random.choice(range(2,7),1)[0])

# only plot detections in the inner region of the patch
keepInner = True
# only plot detections brighter than keepMag
keepMag = 27#False

# ds9 frame number
displayFrameNb = 0

# where to read masks from
#mask_path = '/datasets/hsc/BrightObjectMasks/GouldingMasksS18A/{}/BrightObjectMask-{}-{}-{}.reg'.format(
#            tract,tract,patch,chosenFilter) # Andy's masks
mask_path = "/datasets/hsc/BrightObjectMasks/ArcturusMasks/{}/BrightObjectMask-9697-{}-{}.reg".format(
            tract, patch, chosenFilter) # Arcturus masks (note they're all the I band ones)

## Set up

In [2]:
%matplotlib inline
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
from lsst import geom
import os
import helperFuncs

In [3]:
butler = Butler("/project/mschmitz/hscRerun/mynobg")

## Show on display
Start by firing it up:

In [4]:
coadd_fn = butler.get('deepCoadd_calexp_filename', tract=tract, patch=patch, 
                      filter=chosenFilter)
print(coadd_fn)
if not os.path.exists(coadd_fn[0]):
    print('You are probably reading from parent; worth double checking!')

coadd = butler.get('deepCoadd_calexp', tract=tract, patch=patch, filter=chosenFilter)

['/project/mschmitz/hscRerun/mynobg/deepCoadd-results/HSC-R/9697/2,3/calexp-HSC-R-9697-2,3.fits']


In [5]:
display = afwDisplay.Display(displayFrameNb)
mask = coadd.getMask()
display.mtv(coadd)
display.scale("linear", "zscale")
display.setMaskTransparency(100)

Load detections and mask info:

In [6]:
src_fn = butler.get('deepCoadd_meas_filename', tract=tract, patch=patch, filter=chosenFilter)
if not os.path.exists(src_fn[0]):
    print('You are probably reading from parent; worth double checking!')
#src = butler.get('deepCoadd_ref', tract=tract, patch=patch, filter=chosenFilter)
allSrc = butler.get('deepCoadd_meas', tract=tract, patch=patch, filter=chosenFilter)
print(len(allSrc))

30743


Filter kept detections using specified criteria:

In [7]:
src = allSrc.copy()
if keepInner:
    src = src[src['detect_isPatchInner']]

if keepMag:
    photoCalib = coadd.getPhotoCalib()
    ABmags = photoCalib.instFluxToMagnitude(src, 'base_PsfFlux')
    src = src[ABmags[:,0] < keepMag]
    
print(len(src),len(allSrc))

19529 30743


In [8]:
# get sky map and WCS for this tract
skymap = butler.get("deepCoadd_skyMap", tract=tract, filter=chosenFilter)
tractInfo = skymap[tract]
wcs = tractInfo.getWcs()
pixScale = geom.radToArcsec(wcs.getPixelScale())

In [9]:
brightCenters, brightMags, brightRadii = helperFuncs.extractBrightStarInfo(tractInfo,
                                                                           mask_path)

And plot them on display. (Note you can check these either by loading in Andy's .reg file, or comparing with the mask plane for bright objects):

In [10]:
with display.Buffering():
    for j,cpix in enumerate(brightCenters):
        display.dot("o", cpix[0], cpix[1], size=brightRadii[j],
                    ctype='orange')

Determine detections within that patch and outside all bright star masks:

In [11]:
insideCents = np.array([obj.getCentroid() for obj in src])

In [12]:
outsideMask = np.ones(len(insideCents)).astype(bool)
for brightCen, pixRad in zip(brightCenters, brightRadii):
    dist = np.sqrt(np.sum((insideCents - brightCen)**2, axis=1))
    outsideMask[dist < pixRad] = False

And now plot them, see if you get something like the known problem plot:

In [13]:
with display.Buffering():
    for cent in insideCents[outsideMask]:
        display.dot("o", cent[0], cent[1], 
                    size=5, ctype='green')